In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from collections import namedtuple
from os import listdir

Entry = namedtuple('Entry',['value','cluster'])

In [ ]:
files =  listdir('/Users/dgrossman/data/')

In [ ]:
filelist = list()
for f in files:
    if f.endswith('.out') and f.startswith('tbirdOverlap'):
        filelist.append(f)

In [ ]:
filelist

In [ ]:
documents = dict()
for f in filelist:
    a = open('/Users/dgrossman/data/' + f,'r')
    words = ""
    for w in a.readlines():
        words += w.lstrip().strip()
        words += ' '
    a.close()
    documents[f] = words.strip()

In [ ]:
doc_set = list()
for d in documents.itervalues():
    doc_set.append(d)


In [ ]:
texts = []
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
for d in doc_set:
    tokens = tokenizer.tokenize(d)
    texts.append(tokens)

In [ ]:

dictionary = corpora.Dictionary(texts)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=200)

In [ ]:
outData = list()
for x in(ldamodel.print_topics(num_topics=10, num_words=40)):
    parts =  x[1].split('+')
    sum = 0
    lines = list()
    empty = True
    # print x
    for p in parts:
        val,cluster = p.lstrip().strip().split('*')
        # print val,cluster
        
        if (sum + float(val) < .8) or empty:
            sum += float(val)
            # print 'adding:',cluster
            empty = False
            lines.append(Entry(float(val),int(cluster)))
    #print sum,lines
    #print
    outData.append((sum,lines))
            

In [ ]:
import re
templateFile = '/Users/dgrossman/data/tbird.Clusters'
tf = open(templateFile,'r').readlines()
templateList = list()
for t in tf:
    # print t.strip()
    unescaped= re.sub(r'[\^]','',re.sub(r'[\\]', '', t)).strip()
    templateList.append(unescaped)


In [ ]:
for topic, X in enumerate(outData):
    s = X[0]
    ent = X[1]
    print 'topic=%i|count=%i|energy=%f' % (topic, len(ent), s)
    wordBag = set()
    for e in ent:
        cluster, string = templateList[e.cluster].split(',',1)
        print '\t%5i| %1.4f| %s' % (int(cluster),e.value,string)
        for s in string.split():
            wordBag.add(s)
    print
    print 'words used : %s' % ' '.join(sorted(wordBag))
    print
    